# Building a spam filter with Naive Bayes algorithm.

In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm.

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by *Tiago A. Almeida and José María Gómez Hidalgo*, and it can be downloaded from the The [University of California at Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/228/sms+spam+collection).

This project is based on and extends a similar project shared by [mircealex @ DataQuest](https://github.com/dataquestio/solutions/blob/master/Mission433Solutions.ipynb).

We will use **pandas** to manipluate the data. A multinomial naive Bayes algorithm is used and the mathematical details are given below in the notebook.

In [2]:
import pandas as pd
import re

## Exploring the Dataset

First we read in the dataset and do basic checks - size, head, tail.

In [3]:
sms_data = pd.read_csv('data/SMSSpamCollection', sep='\t', header=None, names = ['Label', 'SMS'])
print(sms_data.shape)
print( sms_data.head() )
print( sms_data.tail() )

(5572, 2)
  Label                                                SMS
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
     Label                                                SMS
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham               Will ü b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name


In the head/tail we see that most messages are labelled as ham, some are spam. This is normal. Let's check the exact percentages.

In [4]:
sms_data['Label'].value_counts(normalize=True)

Label
ham     0.865937
spam    0.134063
Name: proportion, dtype: float64

We see above that 86.6% messages are ham, and 13.4% are spam. This sample looks representative based on common experience.

# Working with the data
## Training/Test split
Now we split the data into training and test sets (the usual 80/20 split). 80% for the training and 20% for testing

In [5]:
# First randomise the dataset
data_randomised = sms_data.sample(frac = 1, random_state= 1)

# Calculate index for split at 80% of data
split_index = round(len(data_randomised) * 0.8 )

# Training/ test split at the split_index
training_data = data_randomised[ : split_index ].reset_index(drop=True)
test_data = data_randomised[ split_index: ].reset_index(drop=True)

print("Shape of training data is ", training_data.shape, " and the shape of test data is ", test_data.shape)

# Now do a sanity check to see that both sets of data has the ham/spam ratio of the original data.
print ("Training set: \n", training_data['Label'].value_counts(normalize=True) )
print ("Test set: \n", test_data['Label'].value_counts(normalize=True) )

Shape of training data is  (4458, 2)  and the shape of test data is  (1114, 2)
Training set: 
 Label
ham     0.86541
spam    0.13459
Name: proportion, dtype: float64
Test set: 
 Label
ham     0.868043
spam    0.131957
Name: proportion, dtype: float64


The ratios look similar to the original dataset, which his good. We now move on to clean the dataset.

## Data cleaning
In order to work with the algorithm for calculating probabilities, we need to convert all the text messages to see the number of keywords. The below image captures the essence of what we need to do.

![](data_cleaning.png)

### Remove punctuation and convert everything to lower case

In [6]:
# Remove all the punctuation and bring all letters to lower case
print("Data before cleaning: \n", training_data.head())

training_data['SMS'] = training_data['SMS'].str.replace(r'\W', ' ', regex=True) # replace any non-word characters with a space
training_data['SMS'] = training_data['SMS'].str.lower() # convert everything to lower case

print("Data after cleaning: \n", training_data.head())

# print(training_data['SMS'].str.replace(r'\W', ' ', regex=True).head())

Data before cleaning: 
   Label                                                SMS
0   ham                       Yep, by the pretty sculpture
1   ham      Yes, princess. Are you going to make me moan?
2   ham                         Welp apparently he retired
3   ham                                            Havent.
4   ham  I forgot 2 ask ü all smth.. There's a card on ...
Data after cleaning: 
   Label                                                SMS
0   ham                       yep  by the pretty sculpture
1   ham      yes  princess  are you going to make me moan 
2   ham                         welp apparently he retired
3   ham                                            havent 
4   ham  i forgot 2 ask ü all smth   there s a card on ...


### Create a vocabulary of all the unique words

In [7]:
training_data['SMS'] = training_data['SMS'].str.split() # this converts SMS column into a list of all the words

vocabulary = []

for sms in training_data['SMS']:
    for word in sms:
        vocabulary.append(word)

# Currently vocabulary contains all the words in the dataset. Remove duplicates:
vocabulary = set(vocabulary)
print("The total number of words in our vocabulary is", len(vocabulary))


The total number of words in our vocabulary is 7783


In [8]:
# First initialise a dictionary with zeros. Each word corresponds to a list of numbers of how many times it appears in each sms.
word_count_per_sms = {unique_word: [0]* len(training_data['SMS']) for unique_word in vocabulary }

# This loop assigns the current frequency of each word.
for index, sms in enumerate(training_data['SMS']):
    for word in sms:
        word_count_per_sms[word][index] +=1 

# Now convert this dictionary to a dataframe
word_counts = pd.DataFrame(word_count_per_sms)
word_counts.head()


,doit,unintentional,everyday,den,christmas,unspoken,def,remain,bcs,secret,...,jada,doubletxt,shah,agree,december,tddnewsletter,offered,6zf,carefully,tix
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Now merge the above with the modified training set dataframe to get the final dataset to work with
training_data_clean = pd.concat([training_data, word_counts], axis= 1)
training_data_clean.head()

,Label,SMS,doit,unintentional,everyday,den,christmas,unspoken,def,remain,...,jada,doubletxt,shah,agree,december,tddnewsletter,offered,6zf,carefully,tix
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Application of the Multinomial Naive Bayes algorithm

For each new message, we need to calculate the following probabilities to classify as spam or ham.

\begin{equation}
P(Spam| w_1, w_2, \dotsc , w_n) \propto P (Spam)  \prod\limits_{i=1}^{n} P(w_i | Spam )
\end{equation}

\begin{equation}
P(Ham| w_1, w_2, \dotsc , w_n) \propto P (Ham)  \prod\limits_{i=1}^{n} P(w_i | Ham )
\end{equation}

In order to calculate the conditional probabilities $ P(w_i | Spam )$ and $P(w_i | Ham )$, we will use the following standard formulas:

\begin{equation}
P(w_i | Spam ) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In the above:

- $N_{w_i|Spam}$ = number of times the word $w_i$ appears in spam messages.
- $N_{w_i|Ham}$ = number of times the word $w_i$ appears in ham messages.
- $N_{Spam}$ = total number of words in the Spam messages.
- $N_{Ham}$ = total number of words in the Ham messages.
- $N_{Spam}$ = number of words in the total vocabulary.
- $\alpha$ = Laplace smoothing parameter (default is 1).

We can directly use our training set to calculate $N_{Spam}$, $N_{Ham}$, $N_{Vocabulary}$, $P (Spam)$ and $P (Ham)$.

In [10]:
# Isolate spam and ham messages into different datasets
spam_data = training_data_clean[training_data_clean['Label'] == 'spam']
ham_data = training_data_clean[training_data_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_data)/ len(training_data_clean)
p_ham = 1 - p_spam

# N_Spam
n_per_spam = spam_data['SMS'].apply(len)
n_spam = n_per_spam.sum()

# N_Ham
n_per_ham = ham_data['SMS'].apply(len)
n_ham = n_per_ham.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing default value
alpha = 1

print("The obtained variables are: P(Spam) =", round(p_spam, 2), ", P(Ham) = ", round(p_ham, 2), 
      ", N_Spam =", n_spam, ", N_Ham =", n_ham, ", N_Vocabulary =", n_vocabulary) 

The obtained variables are: P(Spam) = 0.13 , P(Ham) =  0.87 , N_Spam = 15190 , N_Ham = 57237 , N_Vocabulary = 7783


### Conditional probabilities

Having calculated the variables above, we can now calculate the conditional probability  $ P(w_i | Spam )$ and $P(w_i | Ham )$ for each word using the formulas stated above. The only new calculation is to find $N_{w_i|Spam}$ and $N_{w_i|Ham}$ for each word.

In [11]:
# Initiate the variables for conditional probabilities in a dictionary data structure.
probabilities_spam = {unique_word: 0 for unique_word in vocabulary}
probabilities_ham = {unique_word: 0 for unique_word in vocabulary}

# Assign the actual values
for word in vocabulary:
    n_word_given_spam = spam_data[word].sum()
    probabilities_spam[word] = ( n_word_given_spam + alpha ) / ( n_spam + alpha* n_vocabulary )

    n_word_given_ham = ham_data[word].sum()
    probabilities_ham[word] = ( n_word_given_ham + alpha ) / ( n_ham + alpha* n_vocabulary )


# Classifying a new message

Now that all the background variables are calculated, we can start creating the spam filter. The function works as follows:

- Input a new message $(w_1, w_2, \dotsc , w_n)$.
- Calculate $P(Spam | w_1, w_2, , \dotsc, w_n)$ and $P(Ham | w_1, w_2, , \dotsc, w_n)$ using the formulas listed above.
- Use the above probabilities to classify as ham or spam.
    - Ham if $P(Spam | w_1, w_2, , \dotsc, w_n) < P(Ham | w_1, w_2, , \dotsc, w_n)$
    - Spam if $P(Spam | w_1, w_2, , \dotsc, w_n) > P(Ham | w_1, w_2, , \dotsc, w_n)$
    - Undecided if $P(Spam | w_1, w_2, , \dotsc, w_n) = P(Ham | w_1, w_2, , \dotsc, w_n)$ (although this is expected to be quite rare)

In [12]:
# Define the classifier function
def classify(sms):
    '''
    Input the sms as a string.
    '''

    # Remove punctuation and convert to lower case
    sms = re.sub('\W', ' ', sms)
    sms = sms.lower().split()

    # initialise the probabilities
    p_spam_given_sms = p_spam
    p_ham_given_sms = p_ham

    for word in sms:
        if word in probabilities_spam:
            p_spam_given_sms *= probabilities_spam[word]
        
        if word in probabilities_ham:
            p_ham_given_sms *= probabilities_ham[word]

    print('P(Spam|sms) =', p_spam_given_sms, 'and P(Ham|sms) =', p_ham_given_sms)
    
    if p_ham_given_sms > p_spam_given_sms:
        print('Label: Ham')
    elif p_spam_given_sms > p_ham_given_sms:
        print('Label: Spam')
    else:
        print('Equal probabilities. A human should classify this.')


In [13]:
classify('Alright then, see you later')

P(Spam|sms) = 7.538598683744844e-19 and P(Ham|sms) = 5.625552349491667e-14
Label: Ham


In [14]:
classify('Winner!! Click here to win the prize')

P(Spam|sms) = 9.654365138458546e-22 and P(Ham|sms) = 5.159017989360921e-25
Label: Spam


In [15]:
classify('You have been chosen for the jackpot')

P(Spam|sms) = 8.007841742970287e-21 and P(Ham|sms) = 2.6533008318723246e-21
Label: Spam


## Analyse the filter

The filter works well for the manual examples above. But let's define a proper way to assess its accuracy. Modify the above function to return a classification label rather than printing the result. Then apply that on the test data.

In [16]:
# Define the classifier function
def classify_test_data(sms):
    '''
    Input the sms as a string.
    '''

    # Remove punctuation and convert to lower case
    sms = re.sub('\W', ' ', sms)
    sms = sms.lower().split()

    # initialise the probabilities
    p_spam_given_sms = p_spam
    p_ham_given_sms = p_ham

    for word in sms:
        if word in probabilities_spam:
            p_spam_given_sms *= probabilities_spam[word]
        
        if word in probabilities_ham:
            p_ham_given_sms *= probabilities_ham[word]

    # print('P(Spam|sms) =', p_spam_given_sms, 'and P(Ham|sms) =', p_ham_given_sms)
    
    if p_ham_given_sms > p_spam_given_sms:
        return 'ham'
    elif p_spam_given_sms > p_ham_given_sms:
        return 'spam'
    else:
        return 'undecided'

In [17]:
# Use the above function on the test set to create a new column of predicted values
test_data['predictions'] = test_data['SMS'].apply(classify_test_data)
test_data.head()

,Label,SMS,predictions
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [18]:
# Measure the accuracy

correct_predictions = 0
total_rows = test_data.shape[0]

for _ , row in test_data.iterrows():
    if row['Label'] == row['predictions']:
        correct_predictions +=1

print ("Number of correct predictions:", correct_predictions, "out of a total", total_rows, "messages.")
print ("Accuracy is", round( 100* correct_predictions/ total_rows , 2), "%.")

Number of correct predictions: 1100 out of a total 1114 messages.
Accuracy is 98.74 %.


The accuracy is over 98% which is pretty good. Now we look at the messages below to see where did our classifier fail.

In [27]:
# Print the ham messages that we classified incorrectly.

print ("Messages incorrectly flagged as spam: \n")
for _, row in test_data.iterrows():
    if row['Label'] != row['predictions'] and row['Label'] == 'ham':
        print(row['SMS'])

print ("\n \n Spam messages that were missed: \n")
for _, row in test_data.iterrows():
    if row['Label'] != row['predictions'] and row['Label'] == 'spam':
        print(row['SMS'])

Messages incorrectly flagged as spam: 

Unlimited texts. Limited minutes.
26th OF JULY
Nokia phone is lovly..
A Boy loved a gal. He propsd bt she didnt mind. He gv lv lttrs, Bt her frnds threw thm. Again d boy decided 2 aproach d gal , dt time a truck was speeding towards d gal. Wn it was about 2 hit d girl,d boy ran like hell n saved her. She asked 'hw cn u run so fast?' D boy replied "Boost is d secret of my energy" n instantly d girl shouted "our energy" n Thy lived happily 2gthr drinking boost evrydy Moral of d story:- I hv free msgs:D;): gud ni8
No calls..messages..missed calls
We have sent JD for Customer Service cum Accounts Executive to ur mail id, For details contact us

 
 Spam messages that were missed: 

Not heard from U4 a while. Call me now am here all night with just my knickers on. Make me beg for it like U did last time 01223585236 XX Luv Nikiyu4.net
More people are dogging in your area now. Call 09090204448 and join like minded guys. Why not arrange 1 yourself. There'

### Another definition of accuracy
Another way to analyse the accuracy is by looking at the percentage of spam messages the filter misses and the percentage of ham messages it mislabels and blocks.

In [29]:
# number of ham/spam in test data
num_ham_test = test_data['Label'].value_counts()['ham']
num_spam_test = test_data['Label'].value_counts()['spam']

num_ham_correct_pred = 0
num_spam_correct_pred = 0

for _, row in test_data.iterrows():
    if row['Label'] == 'ham':
        if row['predictions'] == 'ham':
            num_ham_correct_pred += 1
    elif row['Label'] == 'spam':
        if row['predictions'] == 'spam':
            num_spam_correct_pred += 1

print ("Of a total", num_ham_test, "ham messages,", num_ham_correct_pred, "were identified correctly.")
print ("of a total", num_spam_test, "spam messages,", num_spam_correct_pred, "were identified correctly." )

# Calculate the sensitivity and specificity of the test.
# Sensitivity = probability of the message being declared a spam given that it is a spam.
print ("Sensitivity of the filter is", round (100* num_spam_correct_pred/ num_spam_test, 2), "%.")

# Specificity = probability of the message being declared ham given that it is ham.
print ("Specificity of the filter is", round (100* num_ham_correct_pred/ num_ham_test, 2), "%.")

Of a total 967 ham messages, 961 were identified correctly.
of a total 147 spam messages, 139 were identified correctly.
Sensitivity of the filter is 94.56 percent.
Specificity of the filter is 99.38 percent.


## Ways to improve...

Accuracy is currently over 98% but it can also be improved further. 

The following parameters have been manually used in the entire code and changing them changes the final result:
- percentage of data split between training and test set (currently 80%)
- random_state while randomising the data (current value = 1)
- Laplace smoothing parameter (current value = 1)

Other possibilities:
- Would it be useful to keep upper and lower cases in the messages? It will increase the size of vocabulary and may make the algorithm more accurate.
- More complex algorithm using Machine Learning. Currently we're just looking at individual word frequencies without any context. Using a more refined NLP code will make this better - although that's a completely different project.